In [91]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.spatial.distance import cdist

In [92]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Suppress ConvergenceWarnings
warnings.simplefilter(action='ignore', category=ConvergenceWarning)


In [93]:
df= pd.read_csv('PUBLIC REMARKS INCLUDE STR LANGUAGE.csv', low_memory=False)

In [94]:
df.info(max_cols=180, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8254 entries, 0 to 8253
Data columns (total 161 columns):
 #    Column                                  Non-Null Count  Dtype  
---   ------                                  --------------  -----  
 0    List Number                             8254 non-null   int64  
 1    Agency Name                             8254 non-null   object 
 2    Agency Phone                            8212 non-null   object 
 3    Listing Agent                           8254 non-null   object 
 4    Co-Listing Agent                        2327 non-null   object 
 5    Property Type                           8254 non-null   object 
 6    Card Format                             8254 non-null   object 
 7    Selling Agency                          4687 non-null   object 
 8    Selling Agent                           4687 non-null   object 
 9    Co-Selling Agent                        33 non-null     object 
 10   End Date                                0 non-

In [95]:
df = df.drop_duplicates()
df1=df.copy()

In [96]:
def drop_columns_with_high_null(df, threshold=0.75):
    null_percentages = df.isnull().mean()
    columns_to_drop = null_percentages[null_percentages > threshold].index
    df = df.drop(columns=columns_to_drop)
    return df

In [97]:
df=drop_columns_with_high_null(df, threshold=0.67)

In [98]:
df.isnull().sum().sort_values()

List Number                0
Geo Lat                    0
Geo Lon                    0
Year Built                 0
Buyer/Broker               0
                        ... 
Close of Escrow Date    3566
Selling Agency          3567
Selling Agent           3567
Comp to Subagent        3902
Approx Lot Acres        4629
Length: 76, dtype: int64

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8254 entries, 0 to 8253
Data columns (total 76 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   List Number                 8254 non-null   int64  
 1   Agency Name                 8254 non-null   object 
 2   Agency Phone                8212 non-null   object 
 3   Listing Agent               8254 non-null   object 
 4   Property Type               8254 non-null   object 
 5   Card Format                 8254 non-null   object 
 6   Selling Agency              4687 non-null   object 
 7   Selling Agent               4687 non-null   object 
 8   Dwelling Type               8254 non-null   object 
 9   List Date                   8254 non-null   object 
 10  Close of Escrow Date        4688 non-null   object 
 11  Under Contract Date         5584 non-null   object 
 12  Status                      8254 non-null   object 
 13  Status Change Date          8254 

In [100]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
List Number,8254.0,NaN,NaN,NaN,5737130.944996,738313.967584,2584621.0,5061433.5,5109077.5,6574456.5,6603510.0
Agency Name,8254,1502,HomeSmart (cril01),191,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Agency Phone,8212,1352,(602) 230-7600,493,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Listing Agent,8254,4890,Ashley Pickens,136,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Type,8254,1,Residential,8254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Dwelling Styles,8169,3,Detached,7027,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Approx Lot Acres,3625.0,NaN,NaN,NaN,1.890366,32.550135,0.0,0.124,0.171,0.358,1739.0
Photo URL,8242,8242,http://cdn.photos.sparkplatform.com/az/2023082...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Days on Market,8254.0,NaN,NaN,NaN,77.808941,98.565867,0.0,19.0,47.0,102.0,1627.0


In [101]:
constant_columns = [col for col in df.columns if df[col].nunique() == 1]

# Printing the list of constant columns
print("Columns with constant value:", constant_columns)


Columns with constant value: ['Property Type', 'Card Format']


### These columns ['Property Type', 'Card Format'] are all RESIDENTIAL properties, sio droppping them for better Data Handlbng

In [102]:
df.drop('Property Type',axis=1, inplace= True )
df.drop('Card Format',axis=1, inplace= True )

In [103]:
df['Public Remarks']

0       Discover the perfect blend of comfort, conveni...
1       THIS IS YOUR NEXT HOME! Move-In Ready, and rar...
2       Comfortable and stylish living space awaits yo...
3       1+ACRE property in desirable Huachuca Mountain...
4       This extremely sought-after, newly constructed...
                              ...                        
8249    Original owner selling energy efficient home w...
8250    Unique 3 bedroom 2 bath home on 3.42 acres of ...
8251    Maracay Single Level home in the highly desira...
8252    This gorgeous new home features  granite kitch...
8253    Beautiful new single level home located in the...
Name: Public Remarks, Length: 8254, dtype: object

In [104]:

# Creating a boolean mask for each word pattern we're interested in
mask_str = df['Public Remarks'].str.contains(r'\bstr\b', case=False, na=False, regex=True)
mask_short_term_rental = df['Public Remarks'].str.contains(r'\bshort term rental\b', case=False, na=False, regex=True)
mask_vacation_rental = df['Public Remarks'].str.contains(r'\bVacation rental\b', case=False, na=False, regex=True)



In [105]:
mask_str.sum()

22

In [106]:
mask_short_term_rental.sum()

72

In [107]:
mask_vacation_rental.sum()

97

In [108]:
# Combining these masks with 'or' (|)
final_mask = mask_str | mask_short_term_rental | mask_vacation_rental

# Filtering the DataFrame based on the final mask
filtered_df = df[final_mask]

filtered_df

,List Number,Agency Name,Agency Phone,Listing Agent,Selling Agency,Selling Agent,Dwelling Type,List Date,Close of Escrow Date,Under Contract Date,...,Assessor's Map #,Assessor's Parcel #,Off Market Date,Cross Street,Subdivision,Dwelling Styles,Approx Lot Acres,Photo URL,Days on Market,Features
54,6575418,RE/MAX Sedona (rmxs01),(928) 282-4166,Roy Grimm,NaN,NaN,Single Family - Detached,2023-06-30,NaN,NaN,...,44.0,8.0,NaN,Bristlecone Pines Rd.,Las Lomas,Detached,0.861,http://cdn.photos.sparkplatform.com/az/2023051...,72,Association & Fees|HOA Y/N|N;Association & Fee...
63,6567648,NORTH&CO. (nrtc01),(602) 714-7000,Liza Czopp,NaN,NaN,Townhouse,2023-06-07,NaN,NaN,...,36.0,25.0,NaN,Scottsdale Road and Camelback,COLONY CAMELBACK,Attached,0.001,http://cdn.photos.sparkplatform.com/az/2023061...,91,Association & Fees|HOA Paid Frequency|Monthly;...
80,6579707,Keller Williams Realty East Valley (kwevt01),(480) 839-6600,Leonard Saavedra,NaN,NaN,Single Family - Detached,2023-07-06,NaN,NaN,...,8.0,72.0,NaN,From HWY 87 ( Beeline Hwy ) and W.Wade Lane,PAYSON HEIGHTS,Detached,0.239,http://cdn.photos.sparkplatform.com/az/2023070...,62,Association & Fees|HOA Y/N|N;Items Updated|Flo...
81,6582692,NORTH&CO. (nrtc01),(602) 714-7000,Sheryl Matuszak,NaN,NaN,Single Family - Detached,2023-07-20,NaN,NaN,...,7.0,9.0,NaN,Cactus and 58th Way,AVANT 1 UNIT 4,Detached,0.415,http://cdn.photos.sparkplatform.com/az/2023071...,52,Association & Fees|HOA Y/N|N;Items Updated|Poo...
85,6587878,Arizona Elite Properties (ROCK16),(928) 476-2973,Evelyn Beck,NaN,NaN,Single Family - Detached,2023-08-03,NaN,NaN,...,69.0,208.0,NaN,"Hwy. 87 & Anasazi in Pine, AZ",Woodshire at Elk Rim,Detached,0.137,http://cdn.photos.sparkplatform.com/az/2023080...,38,Association & Fees|HOA Paid Frequency|Quarterl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7130,5045673,Realty Executives (reax09),(480) 948-9450,J. Garry Folino,RE/MAX Professionals (rmxi02),Mary Groff,Apartment Style/Flat,2013-12-24,2014-05-30,2014-05-11,...,73.0,660.0,2014-05-30,92ND STREET & RAINTREE,THE ALLISON,Stacked,NaN,http://cdn.photos.sparkplatform.com/az/2013122...,157,Association & Fees|HOA Paid Frequency|Monthly;...
7133,5110883,Sandra Wilken Properties (swlp01),(480) 596-0001,Gregg Hayes,Sandra Wilken Properties (swlp01),Gregg Hayes,Single Family - Detached,2014-05-05,2014-05-30,2014-05-05,...,27.0,44.0,2014-05-30,Lincoln Drive & 52nd Place,MacDonald Acres - Exception,Detached,NaN,http://cdn.photos.sparkplatform.com/az/2014042...,25,Association & Fees|Rec Center Fee Y/N|N;Associ...
7281,5104609,HomeSmart (cril10),(602) 230-7600,Andrea Ann Loukota,US Preferred Realty (uspr07),Alex Rabinovich,Townhouse,2014-04-23,2014-06-02,2014-05-02,...,25.0,74.0,2014-05-31,Hayden & Thomas,Scottsdale Villas,Attached,NaN,http://cdn.photos.sparkplatform.com/az/2014042...,38,Association & Fees|HOA Paid Frequency|Monthly;...
8185,5110725,Southwest Preferred Properties (swpp01),(602) 266-0500,Susan E Fix,Southwest Preferred Properties (swpp01),Susan E Fix,Single Family - Detached,2014-05-05,2014-06-27,2014-05-19,...,87.0,16.0,2014-05-24,"EXIT 126 N TO PEBBLECREEK ENTRANCE,LEFT",PEBBLECREEK UNIT 9,Detached,NaN,http://cdn.photos.sparkplatform.com/az/2014020...,19,Association & Fees|HOA Paid Frequency|Semi-Ann...


In [109]:
filtered_df['Public Remarks']

54      Flagship luxury Vacation Rental property.  Tur...
63      LOCATION, LOCATION!! This beautifully remodele...
80      COMPLETELY REMODELED & PERMITTED, Move in read...
81      Modern, 5 bedrooms, 3 baths remodeled and full...
85      Come see this Beautiful Country-Modern Home in...
                              ...                        
7130    ONE OF THE MOST POPULAR COMPLEXES IN N.SCOTTSD...
7133    EXCEPTIONAL PARADISE VALLEY LOCATION!   MAGNIF...
7281    Don't miss out on this well priced townhouse i...
8185    This fantastic buy includes everything!  All t...
8216    Almost 1/4 acre corner lot one street from gol...
Name: Public Remarks, Length: 184, dtype: object

## Comments :
### * There are some inputs like STR - No , etc which is also included in this filtered_df which needed to be handled. 
### * So we have around only 150 rows/input  in the final dataframe among 8000+ rows 

In [110]:
x=df1[df1['City/Town Code']=='Phoenix']

In [111]:
x['City/Town Code']

2       Phoenix
9       Phoenix
12      Phoenix
13      Phoenix
17      Phoenix
         ...   
8234    Phoenix
8235    Phoenix
8238    Phoenix
8241    Phoenix
8249    Phoenix
Name: City/Town Code, Length: 1990, dtype: object

In [112]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1990 entries, 2 to 8249
Columns: 161 entries, List Number to Unnamed: 160
dtypes: float64(55), int64(7), object(99)
memory usage: 2.5+ MB


In [113]:
x=drop_columns_with_high_null(df, threshold=0.98)

### We have 1990 entries in Phoenix

In [114]:
x.rename(columns={'Geo Lat': 'Latitude'}, inplace=True)
x.rename(columns={'Geo Lon': 'Longitude'}, inplace=True)
x.rename(columns={'# Bedrooms': 'Bedrooms'}, inplace=True)

In [115]:
x.info(max_cols=180, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8254 entries, 0 to 8253
Data columns (total 74 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   List Number                 8254 non-null   int64  
 1   Agency Name                 8254 non-null   object 
 2   Agency Phone                8212 non-null   object 
 3   Listing Agent               8254 non-null   object 
 4   Selling Agency              4687 non-null   object 
 5   Selling Agent               4687 non-null   object 
 6   Dwelling Type               8254 non-null   object 
 7   List Date                   8254 non-null   object 
 8   Close of Escrow Date        4688 non-null   object 
 9   Under Contract Date         5584 non-null   object 
 10  Status                      8254 non-null   object 
 11  Status Change Date          8254 non-null   object 
 12  Original List Price         8254 non-null   float64
 13  List Price                  8254 

In [116]:
df2= pd.read_csv('Phoenix_data_USD (1).csv', low_memory=False)

In [117]:
df2.info(max_cols=180, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7035 entries, 0 to 7034
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Listing URL             7035 non-null   object 
 1   Property Type           7035 non-null   object 
 2   Latitude                7035 non-null   float64
 3   Longitude               7035 non-null   float64
 4   Star Rating             7035 non-null   float64
 5   Number of Active Days   7035 non-null   int64  
 6   Bedrooms                7035 non-null   object 
 7   Has pool                7035 non-null   object 
 8   Cleaning Fee            7035 non-null   int64  
 9   Extra Guest Fee         7035 non-null   int64  
 10  Average Daily Rate LTM  7035 non-null   int64  
 11  Occupancy Rate LTM      7035 non-null   int64  
 12  Annual Revenue LTM      7035 non-null   int64  
dtypes: float64(3), int64(6), object(4)
memory usage: 714.6+ KB


In [118]:
df2

,Listing URL,Property Type,Latitude,Longitude,Star Rating,Number of Active Days,Bedrooms,Has pool,Cleaning Fee,Extra Guest Fee,Average Daily Rate LTM,Occupancy Rate LTM,Annual Revenue LTM
0,http://airbnb.com/rooms/61745,Entire Home,33.519190,-112.073470,4.2,3,3,t,350,0,402,40,58804
1,http://airbnb.com/rooms/122773,Entire Home,33.474000,-112.064000,4.9,361,1,f,0,0,106,100,38796
2,http://airbnb.com/rooms/135612,Condo,33.596690,-111.987710,5.0,135,1,t,175,0,92,57,21090
3,http://airbnb.com/rooms/241046,Private Room,33.558070,-112.094420,4.9,367,1,t,20,0,61,72,18249
4,http://airbnb.com/rooms/242595,Private Room,33.529380,-112.096390,4.8,145,1,t,0,10,42,64,9839
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7030,http://airbnb.com/rooms/958723972133502202,Entire Home,33.471815,-112.051348,0.0,13,2,t,0,0,147,100,53802
7031,http://airbnb.com/rooms/958777577132565108,Entire Home,33.383970,-112.012160,0.0,14,Studio,t,0,0,59,100,21594
7032,http://airbnb.com/rooms/959114708267573468,Entire Home,33.384001,-112.166790,0.0,14,4,t,0,0,185,100,67710
7033,http://airbnb.com/rooms/960401395602956462,Entire Home,33.384169,-112.012267,0.0,14,Studio,t,0,0,59,100,21594


In [119]:
df2['Bedrooms'].unique()

array(['3', '1', '5', '8', '4', '2', 'Studio', '9', '6', '0', '10', '7',
       '11'], dtype=object)

In [120]:
df2['Bedrooms'] = df2['Bedrooms'].replace('Studio', '12')
df2['Bedrooms'].unique()

array(['3', '1', '5', '8', '4', '2', '12', '9', '6', '0', '10', '7', '11'],
      dtype=object)

In [121]:
df2['Bedrooms'] = df2['Bedrooms'].astype('int64')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7035 entries, 0 to 7034
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Listing URL             7035 non-null   object 
 1   Property Type           7035 non-null   object 
 2   Latitude                7035 non-null   float64
 3   Longitude               7035 non-null   float64
 4   Star Rating             7035 non-null   float64
 5   Number of Active Days   7035 non-null   int64  
 6   Bedrooms                7035 non-null   int64  
 7   Has pool                7035 non-null   object 
 8   Cleaning Fee            7035 non-null   int64  
 9   Extra Guest Fee         7035 non-null   int64  
 10  Average Daily Rate LTM  7035 non-null   int64  
 11  Occupancy Rate LTM      7035 non-null   int64  
 12  Annual Revenue LTM      7035 non-null   int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 714.6+ KB


In [122]:
merged_df = x.merge(df2, on=['Latitude', 'Longitude'], how='inner')

In [123]:
merged_df

,List Number,Agency Name,Agency Phone,Listing Agent,Selling Agency,Selling Agent,Dwelling Type,List Date,Close of Escrow Date,Under Contract Date,...,Property Type,Star Rating,Number of Active Days,Bedrooms_y,Has pool,Cleaning Fee,Extra Guest Fee,Average Daily Rate LTM,Occupancy Rate LTM,Annual Revenue LTM


In [124]:
merged_df_lat = x.merge(df2, on=['Latitude'], how='inner')
merged_df_lat

,List Number,Agency Name,Agency Phone,Listing Agent,Selling Agency,Selling Agent,Dwelling Type,List Date,Close of Escrow Date,Under Contract Date,...,Longitude_y,Star Rating,Number of Active Days,Bedrooms_y,Has pool,Cleaning Fee,Extra Guest Fee,Average Daily Rate LTM,Occupancy Rate LTM,Annual Revenue LTM
0,6589926,DeLex Realty (dele001),NaN,Carole Wilson,NaN,NaN,Single Family - Detached,2023-08-08,NaN,NaN,...,-112.05236,5.0,195,2,t,0,0,122,55,24560
1,6584262,Russ Lyon Sotheby's International Realty (LYON27),(480) 315-1575,Chauncey Beck,NaN,NaN,Single Family - Detached,2023-07-25,NaN,NaN,...,-112.03348,5.0,351,1,f,50,0,94,55,21425
2,6584262,Russ Lyon Sotheby's International Realty (LYON27),(480) 315-1575,Chauncey Beck,NaN,NaN,Single Family - Detached,2023-07-25,NaN,NaN,...,-112.05819,4.9,151,2,f,0,0,100,68,24888
3,6578874,My Home Group Real Estate (myhg02),(480) 685-2760,Patrick Lee Kaiser,NaN,NaN,Single Family - Detached,2023-07-08,NaN,NaN,...,-112.05049,5.0,323,2,f,20,0,126,65,30928
4,6583548,Libertas Real Estate (liber004),(623) 271-9742,Jeremy Jennings,NaN,NaN,Single Family - Detached,2023-07-22,NaN,NaN,...,-112.07576,5.0,44,4,f,0,0,366,63,84396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,5019569,Keller Williams Realty Sonoran Living (kwsl02),(480) 948-3338,Katie Baccus,Keller Williams Realty Sonoran Living (kwsl02),Katie Baccus,Apartment Style/Flat,2013-10-23,2014-06-24,2014-05-12,...,-112.01278,5.0,295,1,f,65,0,55,48,10146
140,5106020,Better Homes & Gardens Real Estate SJ Fowler (...,(480) 649-3536,Linda Moening,Just Referrals Real Estate (wsrr01),Elliot L Barkan,Single Family - Detached,2014-04-25,2014-06-26,2014-06-17,...,-112.01316,5.0,77,1,t,0,0,94,26,8946
141,5105763,US Preferred Realty (uspr01),(480) 756-9922,Gary Chen,Keller Williams Arizona Realty (kwaz10),Kathleen Ammon,Single Family - Detached,2014-04-25,2014-06-24,2014-06-06,...,-111.96075,4.0,4,1,t,90,0,444,100,162504
142,5116775,Wise Choice Properties (wcpr01),(602) 687-8792,James Carpenter,West USA Realty (wusa06),Asa Hemberg,Single Family - Detached,2014-05-16,2014-06-27,2014-05-20,...,-112.01587,5.0,4,4,t,180,0,491,100,179706


In [125]:
merged_df_lat['Latitude']

0      33.58146
1      33.47328
2      33.47328
3      33.59141
4      33.50717
         ...   
139    33.50555
140    33.38445
141    33.58696
142    33.47214
143    33.37771
Name: Latitude, Length: 144, dtype: float64

In [126]:
merged_df_long = x.merge(df2, on=['Longitude'], how='inner')
merged_df_long

,List Number,Agency Name,Agency Phone,Listing Agent,Selling Agency,Selling Agent,Dwelling Type,List Date,Close of Escrow Date,Under Contract Date,...,Latitude_y,Star Rating,Number of Active Days,Bedrooms_y,Has pool,Cleaning Fee,Extra Guest Fee,Average Daily Rate LTM,Occupancy Rate LTM,Annual Revenue LTM
0,6579754,NORTH&CO. (nrtc01),(602) 714-7000,Michelle K Grening,NaN,NaN,Single Family - Detached,2023-07-13,NaN,NaN,...,33.55807,4.9,367,1,t,20,0,61,72,18249
1,6597498,HomeSmart (cril01),(602) 230-7600,Lisa A Dixon,NaN,NaN,Single Family - Detached,2023-08-25,NaN,NaN,...,33.36519,4.8,46,1,f,43,15,47,34,6874
2,6602869,HomeSmart (cril16),(623) 889-7100,April L Keys,NaN,NaN,Apartment Style/Flat,2023-09-07,NaN,NaN,...,33.46880,5.0,4,1,f,0,0,100,100,36600
3,6602177,West USA Realty (wusa02),(480) 948-5554,AnaMarie Rigo,NaN,NaN,Single Family - Detached,2023-09-07,NaN,NaN,...,33.49405,5.0,84,2,f,0,0,75,47,12902
4,6571588,eXp Realty (xpre01),(888) 897-7821,Aaron Remer,NaN,NaN,Single Family - Detached,2023-06-21,NaN,NaN,...,33.50614,5.0,207,1,t,0,0,210,66,50727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,5105559,Domaine Real Estate (doma001),(480) 688-3888,Jerry A. Humphrey,Realty Executives (reax15),Mary E Murphy,Single Family - Detached,2014-04-23,2014-06-24,2014-05-03,...,33.47072,5.0,4,3,f,250,0,425,100,155550
76,5108556,My Home Group Real Estate (myhg01),(480) 685-2760,Shawna Noel Scheinerman,West USA Realty (wusa05),Ciprian Jivcu,Single Family - Detached,2014-05-01,2014-06-25,2014-05-13,...,33.49439,5.0,13,2,t,0,0,173,74,46848
77,5108988,Keller Williams Realty Phoenix (kwrp001),(480) 768-9333,Carlie Back,Just Referrals Real Estate (wsrr01),Ron Hollingsworth,Single Family - Detached,2014-05-01,2014-06-26,2014-05-26,...,33.55425,4.6,183,3,f,90,10,136,83,45814
78,5116775,Wise Choice Properties (wcpr01),(602) 687-8792,James Carpenter,West USA Realty (wusa06),Asa Hemberg,Single Family - Detached,2014-05-16,2014-06-27,2014-05-20,...,33.44879,5.0,103,2,t,150,0,126,97,47932


In [127]:
merged_df = x.merge(df2, on=['Latitude', 'Longitude'], how='outer')
merged_df

,List Number,Agency Name,Agency Phone,Listing Agent,Selling Agency,Selling Agent,Dwelling Type,List Date,Close of Escrow Date,Under Contract Date,...,Property Type,Star Rating,Number of Active Days,Bedrooms_y,Has pool,Cleaning Fee,Extra Guest Fee,Average Daily Rate LTM,Occupancy Rate LTM,Annual Revenue LTM
0,6597146.0,"W and Partners, LLC (wandp001)",(602) 828-5767,Elijah Celaya,NaN,NaN,Single Family - Detached,2023-08-21,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6586753.0,DRH Properties Inc (drhp01),(480) 483-0006,Stacey Vannasdale,NaN,NaN,Single Family - Detached,2023-07-31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6592065.0,HomeSmart (cril01),(602) 230-7600,Patricia Corbin,NaN,NaN,Apartment Style/Flat,2023-08-11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6583021.0,Long Realty Company (long20),(520) 439-3900,Jocelyn T Lawley,NaN,NaN,Single Family - Detached,2023-07-21,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6564266.0,Symmetry Realty Brokerage (symm001),(928) 779-5700,Deana Keck,NaN,NaN,Patio Home,2023-06-02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Entire Home,0.0,13.0,2.0,t,0.0,0.0,147.0,100.0,53802.0
15285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Entire Home,0.0,14.0,12.0,t,0.0,0.0,59.0,100.0,21594.0
15286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Entire Home,0.0,14.0,4.0,t,0.0,0.0,185.0,100.0,67710.0
15287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Entire Home,0.0,14.0,12.0,t,0.0,0.0,59.0,100.0,21594.0


### We dont see any positive results here
### lets trunacate the Latitude and Logitude to 3 decimals values and proceed next

In [128]:
x['Latitude'] =x['Latitude'].round(3)
x['Longitude'] =x['Longitude'].round(3)

In [129]:
df2['Latitude'] =df2['Latitude'].round(3)
df2['Longitude'] =df2['Longitude'].round(3)

In [132]:
merged_df = x.merge(df2, on=['Latitude', 'Longitude','Bedrooms'], how='inner')
merged_df

,List Number,Agency Name,Agency Phone,Listing Agent,Selling Agency,Selling Agent,Dwelling Type,List Date,Close of Escrow Date,Under Contract Date,...,Listing URL,Property Type,Star Rating,Number of Active Days,Has pool,Cleaning Fee,Extra Guest Fee,Average Daily Rate LTM,Occupancy Rate LTM,Annual Revenue LTM
0,6592065,HomeSmart (cril01),(602) 230-7600,Patricia Corbin,NaN,NaN,Apartment Style/Flat,2023-08-11,NaN,NaN,...,http://airbnb.com/rooms/734881649522242825,Condo,0.0,50,t,0,0,144,67,35312
1,6519192,My Home Group Real Estate (myhg24),(480) 685-2760,Cathy Piccinini,NaN,NaN,Single Family - Detached,2023-02-10,NaN,NaN,...,http://airbnb.com/rooms/15520295,Entire Home,4.9,260,t,25,0,199,100,77163
2,6582692,NORTH&CO. (nrtc01),(602) 714-7000,Sheryl Matuszak,NaN,NaN,Single Family - Detached,2023-07-20,NaN,NaN,...,http://airbnb.com/rooms/681852339181209422,Entire Home,4.8,214,t,0,0,298,26,28358
3,6595764,eXp Realty (xpre08),(888) 897-7821,Steven Brady,NaN,NaN,Townhouse,2023-08-22,NaN,NaN,...,http://airbnb.com/rooms/934734094325147856,Entire Home,0.0,14,t,0,0,106,17,6588
4,6581466,Realty ONE Group (reog01),(480) 315-1240,Josh Hintzen,NaN,NaN,Single Family - Detached,2023-07-17,NaN,NaN,...,http://airbnb.com/rooms/922577070943220448,Entire Home,0.0,14,t,0,0,628,69,158583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,5099171,Biltmore Lifestyles RE Company (brec01),(602) 418-6145,Cynthia J Stockwell,HomeSmart (cril11),Greg M Gallese,Apartment Style/Flat,2014-03-23,2014-06-24,2014-06-03,...,http://airbnb.com/rooms/656532429347419004,Condo,4.7,31,t,0,0,209,89,68076
256,5099171,Biltmore Lifestyles RE Company (brec01),(602) 418-6145,Cynthia J Stockwell,HomeSmart (cril11),Greg M Gallese,Apartment Style/Flat,2014-03-23,2014-06-24,2014-06-03,...,http://airbnb.com/rooms/683312737425624814,Condo,4.8,58,t,0,0,169,56,34637
257,5099171,Biltmore Lifestyles RE Company (brec01),(602) 418-6145,Cynthia J Stockwell,HomeSmart (cril11),Greg M Gallese,Apartment Style/Flat,2014-03-23,2014-06-24,2014-06-03,...,http://airbnb.com/rooms/860340542265828915,Condo,5.0,62,t,0,0,89,64,20850
258,5039691,Realty Executives (reax09),(480) 948-9450,Pamela E Thurnherr,Coldwell Banker Realty (cbrb02),Cindy Neel,Single Family - Detached,2013-12-05,2014-06-24,2014-05-17,...,http://airbnb.com/rooms/864887600656291733,Entire Home,4.8,67,t,0,0,362,86,113941


In [134]:
merged_df.to_csv('250 prop.csv')

In [146]:
desired_latitude = 33.312
desired_longitude= -111.841
result = x[x['Longitude'] == desired_latitude]
result

,List Number,Agency Name,Agency Phone,Listing Agent,Selling Agency,Selling Agent,Dwelling Type,List Date,Close of Escrow Date,Under Contract Date,...,Assessor's Map #,Assessor's Parcel #,Off Market Date,Cross Street,Subdivision,Dwelling Styles,Approx Lot Acres,Photo URL,Days on Market,Features
